In [1]:
# Imports
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup
from lxml import html

In [2]:
# Requesting main page for links
res_links = requests.get('https://www.karaoke-lyrics.net/lyrics/sea-shanty-56697')
res_links.status_code

200

In [3]:
# Creating list of all links
res_links_soup = BeautifulSoup(res_links.content, 'lxml')
links_list_all = res_links_soup.find_all('a', href=True)

In [4]:
# Creating empty links list
links_list = []

# Looping through all links on page and pulling only links to song lyrics
for link in links_list_all:
    if link['href'].find('/lyrics/sea-shanty/') != -1:
        links_list.append(link['href'])
        
# Removing duplicate song lyrics 
links_list = list(set(links_list))

In [5]:
# Creating empyt dictionary to hold lyrics
shanties_dict = {}

In [6]:
# Looping through each link to extract lyrics
for link in links_list:
    # Requestings lyrics page
    res_lyrics = requests.get(f'https://www.karaoke-lyrics.net{link}')
    res_lyrics.status_code
    
    # Putting in a small sleep to limit requests on server to under 100 per minute
    time.sleep(.75)
    
    # Pulling song title from page
    title = BeautifulSoup(res_lyrics.text).title.text
    title = title.replace('Sea Shanty - ', '')
    title = title.replace(' - lyrics', '')

    # Pilling lyrics from page
    lyrics_soup = BeautifulSoup(res_lyrics.content, 'lxml')
    lyrics_raw = lyrics_soup.find_all('span', {'class': 'para_1lyrics_col1'})
    lyrics_raw = [x.text for x in lyrics_raw]

    # Converting list of strings into one string
    lyrics = ''
    lyrics = lyrics.join(lyrics_raw)

    # Appending shanties_dict
    shanties_dict.update({title : lyrics})

In [7]:
# Creating data frame for songs and lyrics
karaoke_lyrics_songs = pd.DataFrame(shanties_dict.items(), columns= ['title', 'lyrics'])

In [8]:
# Writing to csv for use in other scripts
karaoke_lyrics_songs.to_csv('../../data/karaoke_lyrics_songs.csv')